In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt

from scipy.spatial import Delaunay
from scipy.stats import linregress
from scipy.optimize import curve_fit
from getdist import plots, MCSamples
from iminuit import Minuit
from iminuit.cost import LeastSquares

from qubic.lib.Qgps import GPSAntenna
import qubic.lib.Calibration.Qfiber as ft
 
%matplotlib inline

In [ ]:
# If True, allow plots for debug
DEBUG = False

In [ ]:
# Distances during test 1
distance_base_antenna1_test1 = np.array([20, 20, 20, 20, 20, 100, 100, 100, 100, 200, 200, 200, 200])
distance_base_antenna2_test1 = np.array([20, 50, 100, 200, 300, 300, 200, 100, 50, 50, 100, 200, 300])

# Distance during test 2
distance_base_antenna2_test2 = np.array([300, 300, 300, 200, 200, 200, 200, 100, 100, 100, 100])
distance_antenna1_antenna2_test2 = np.array([50, 100, 200, 50, 100, 200, 300, 50, 100, 200, 300])

# Import data

In [ ]:
### Build GPS data file path
data_path = "calsource_orientation.dat"

### Define the distance between the two antennas
distance_between_antennas = 1.3 # Just to initialize GPSAntenna

In [ ]:
gps_antenna = GPSAntenna(data_path, distance_between_antennas)

names = np.array(['North', 'East', 'Down', 'Roll', 'Yaw'])

del data_path, distance_between_antennas

In [ ]:
index_ini = gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([dt.datetime(year=2024, month=12, day=12, hour=8, minute=55, second=0)]))[0]
print('Initial index = ', index_ini)
print('size observation times : ', len(gps_antenna._datetime))

In [ ]:
gps_antenna.plot_gps_data(index_start=index_ini)

# Build observation time indices

In [ ]:
# Array containing the starting time of each configuration
array_hours = np.array([8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11])
array_minutes_begins = np.array([55, 0, 6, 12, 19, 25, 32, 41, 46, 52, 58, 4, 10, 19, 27, 34, 40, 46, 53, 1, 7, 13, 20, 25])

# Array containing the ending time of each configuration, build by substracting 1 minute to the starting time
# In this case, each time correspond to do the ending time of the previous configuration
array_minutes_ending = array_minutes_begins.copy() - 1
array_minutes_ending[1] = 0

# Array containing the seconds of each configuration
array_seconds_begins = np.ones(array_hours.shape, dtype=int) * 59
array_seconds_ending = np.zeros(array_hours.shape, dtype=int)

# Build the array of datetime associated with each configuration, to delimit the time when the antenna are moving or not
array_datetime = np.array([])
index_array = np.array([], dtype=int)
for index in range(array_hours.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=array_hours[index], minute=array_minutes_begins[index], second=array_seconds_begins[index]))
for index in range(1, array_hours.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=array_hours[index], minute=array_minutes_ending[index], second=array_seconds_ending[index]))
array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=11, minute=33, second=0))

for index in range(array_datetime.shape[0]):
    index_array = np.append(index_array, gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([array_datetime[index]])))
    
del array_hours, array_minutes_begins, array_minutes_ending, array_seconds_begins, array_seconds_ending, array_datetime, index

## Test

In [ ]:
if DEBUG:
    for index in range(int(index_array.size/2)):
        plt.figure(figsize=(15, 8))
        plt.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:])
        plt.xlabel('Date')
        plt.ylabel('Angle (rad)')
        plt.title('GPS Roll Angle')
        plt.vlines(gps_antenna._datetime[index_array[index]], 0, 2*np.pi, color='r', linestyle='--')
        plt.vlines(gps_antenna._datetime[index_array[index + int(index_array.size/2)]], 0, 2*np.pi, color='r', linestyle='--')
        plt.show()
        
    del index

In [ ]:
# Plot to verify the previous delimitation
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpN[index_ini:], color = color_r, label = 'North component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpE[index_ini:], color = color_b, label = 'East component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpD[index_ini:], color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:], color = color_a, label = 'Roll angle')
ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.yaw[index_ini:], color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

for obs_index in index_array:
    ax1.vlines(gps_antenna._datetime[obs_index], 3.5, -1.5, 'grey', linestyles='--')
ax1.vlines(gps_antenna._datetime[index_array[0]], 3.5, -1.5, 'r', linestyles='--')
ax1.vlines(gps_antenna._datetime[index_array[0 + int(index_array.shape[0]/2)]], 3.5, -1.5, 'r', linestyles='--')

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c, obs_index

In [ ]:
# Function to remove the movement between each configurations
def only_data(array, indices):
    data_array = np.array([])
    
    for i in range(int(indices.shape[0]/2)):
        data_array = np.append(data_array, array[indices[i]:indices[i+int(indices.shape[0]/2)]])
    
    return data_array

In [ ]:
data_rpN = only_data(gps_antenna.rpN, index_array)
data_rpE = only_data(gps_antenna.rpE, index_array)
data_rpD = only_data(gps_antenna.rpD, index_array)
data_roll = only_data(gps_antenna.roll, index_array)
data_yaw = only_data(gps_antenna.yaw, index_array)

In [ ]:
# Plot to verify the filtering of the time when the antennas are moving
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(data_rpN, color = color_r, label = 'North component')
ax1.plot(data_rpE, color = color_b, label = 'East component')
ax1.plot(data_rpD, color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(data_roll, color = color_a, label = 'Roll angle')
ax2.plot(data_yaw, color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c

# Noise Power Spectrum

## Def useful functions

In [ ]:
timestep = (gps_antenna._datetime[1] - gps_antenna._datetime[0]).microseconds * 1e-6
print("Timestep : ", timestep, "s.")

def get_ps(array):
    """Function to compute the power spectrum of a given array.

    Parameters
    ----------
    array : array_like
        array containing the data to compute the power spectrum of.

    Returns
    -------
    power_spectrum : array_like
        array containing the power spectrum of the input array.
    freq: array_like
        array containing the frequency of the power spectrum.
    """
    N = array.size
    return np.abs(np.fft.rfft(array))**2, np.fft.rfftfreq(N, d=timestep/2)

In [ ]:
power_spectrum, freq = get_ps(gps_antenna.rpN[index_array[0]:index_array[int(index_array.size/2)]])
power_spectrum, freq = power_spectrum[1:], freq[1:]

plt.plot(freq, power_spectrum)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale('log')
plt.yscale('log')
plt.title('Noise Power Spectrum - North')

del power_spectrum, freq

## Fit 1/f + white with minuit

In [ ]:
def get_ps(array):
    """Function to compute the power spectrum of a given array.

    Parameters
    ----------
    array : array_like
        array containing the data to compute the power spectrum of.

    Returns
    -------
    power_spectrum : array_like
        array containing the power spectrum of the input array.
    freq: array_like
        array containing the frequency of the power spectrum.
    """
    N = array.size
    return np.abs(np.fft.rfft(array))**2, np.fft.rfftfreq(N, d=timestep/2)

def noise_model(x, A_white, f_knee, slope):
    return A_white**2 * (1 + np.abs(f_knee/x)**slope)

### Fit using my Gaussian LogLikelihood function with minuit

In [ ]:
def nll_exp(A, f_knee, alpha, f, ps):
    P_model = noise_model(f, A, f_knee, alpha)
    return np.sum(np.log(P_model) + ps / P_model)

def nll_gauss(A, f_knee, alpha, freq, ps, sigma):
    P_model = noise_model(freq, A, f_knee, alpha)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha):
       return nll_gauss(A, f_knee, alpha, binned_freq, binned_ps, sigma=binned_ps_error)

    m = Minuit(nll_wrapper, A=0.1, f_knee=1, alpha=1)
    m.limits['A'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        plt.plot(freq, ps, label="data")
        plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
data = [gps_antenna.rpN, gps_antenna.rpE, gps_antenna.rpD, gps_antenna.roll, gps_antenna.yaw]
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$']

# Fit for each instrumental configurations
for idata in range(len(data)):
    values, errors, chi2_dof= [], [], []
    for index in range(int(index_array.size/2)):
        if index == 0:
            plot = True
        else:
            plot = False
        val, err, red_chi2 = fit_minuit_ll(data[idata][index_array[index]:index_array[index+(int(index_array.size/2))]], nbins=300, plot=plot, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(3, 5, figsize=(15, 10), sharey='row', sharex=True)

n_datasets = len(data)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(3):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
#    ax[1, idata].set_ylim(0, 10)

for idata in range(n_datasets):
    ax[2, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - over f + White.png')
plt.show()

In [ ]:
del ax, chi2_dof, data, err, errors, fig, idata, index, iparam, label , mean_val, n_datasets, n_points, red_chi2, reduced_chi2_loglike, std_val, val, values, x

## Are the parameters scaled with the distance between antennas ?

In [ ]:
params_value_north = np.array(params_values_loglike, dtype=float)[0]
print(params_value_north.shape)

In [ ]:
print("Distance between antennas : ", distance_antenna1_antenna2_test2)
print("Distance between base and antenna 2 : " , distance_base_antenna2_test2)

In [ ]:
discrete_values = [100, 200, 300]
colors = ['#EE6677', '#228833', '#4477AA']  

# Create a 1-row, 2-column subplot figure
fig = make_subplots(
    rows=1, 
    cols=2, 
    # subplot_titles=[
    #     "Noise of the North component data: \(f_{knee}\)",
    #     "Noise of the North component data: \(A_{white}\)"
    # ]
)

# ----------- First subplot: f_{knee} -----------
for val, col in zip(discrete_values, colors):
    mask = (distance_base_antenna2_test2 == val)
    
    x_vals = distance_antenna1_antenna2_test2[mask]
    # Here we take the column for f_{knee} (index 1)
    y_vals = params_value_north[int(distance_base_antenna1_test1.shape[0]):, 1][mask]
    
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='markers',
            marker=dict(size=10, color=col),
            name=f"{val} cm",
                        showlegend=False  # This removes the legend for the second subplot

        ),
        row=1,
        col=1
    )

# ----------- Second subplot: A_{white} -----------
for val, col in zip(discrete_values, colors):
    mask = (distance_base_antenna2_test2 == val)
    
    x_vals = distance_antenna1_antenna2_test2[mask]
    # Here we take the column for A_{white} (index 0)
    y_vals = params_value_north[int(distance_base_antenna1_test1.shape[0]):, 0][mask]
    
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='markers',
            marker=dict(size=10, color=col),
            name=f"{val} cm"
        ),
        row=1,
        col=2
    )

# Update axis labels for each subplot
fig.update_xaxes(title='Distance between antennas (cm)', row=1, col=1, gridcolor='lightgray')
fig.update_xaxes(title='Distance between antennas (cm)', row=1, col=2, gridcolor='lightgray')
fig.update_yaxes(title=r'$f_{knee}$', row=1, col=1, gridcolor='lightgray')
fig.update_yaxes(title=r'$A_{white}$', row=1, col=2, gridcolor='lightgray')

# Update overall layout
fig.update_layout(
    title='Noise parameters of the North component data',
    legend_title='Distance between base and antenna 2',
    width=1500,
    height=900,
    template='plotly_white',   # sets a clean background theme
    paper_bgcolor='white',     # ensures the outer area is white
    plot_bgcolor='white'       # ensures the plotting area is white
)
fig.write_image(f"Noise parameters with distance - over f + white.png")
fig.show()

## Fit 1/f + linear noise

$P(f) = A_{white}^2 ((\frac{f}{f_{knee}})^{\alpha} + (\frac{f_{knee}}{f})^{\beta})$

In [ ]:
def advanced_noise_model(x, A_white, f_knee, alpha, beta):
    return A_white**2 * (1 + x*alpha/f_knee + (f_knee/x)**beta)

In [ ]:
def nll_gauss(A, f_knee, alpha, beta, freq, ps, sigma):
    P_model = advanced_noise_model(freq, A, f_knee, alpha, beta)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha, beta):
       return nll_gauss(A, f_knee, alpha, beta, binned_freq, binned_ps, sigma=binned_ps_error)

    m = Minuit(nll_wrapper, A=0.1, f_knee=1, alpha=1, beta=1)
    m.limits['A'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)
    m.limits['beta'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        plt.plot(freq, ps, label="data")
        plt.plot(freq, advanced_noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
data = [gps_antenna.rpN, gps_antenna.rpE, gps_antenna.rpD, gps_antenna.roll, gps_antenna.yaw]
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$', r'$\beta$']

# Fit for each instrumental configurations
for idata in range(len(data)):
    values, errors, chi2_dof= [], [], []
    for index in range(int(index_array.size/2)):
        if index == 0:
            plot = True
        else:
            plot = False
        val, err, red_chi2 = fit_minuit_ll(data[idata][index_array[index]:index_array[index+(int(index_array.size/2))]], nbins=250, plot=plot, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(len(label), 5, figsize=(15, 10), sharey='row', sharex=True)

n_datasets = len(data)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(len(label)):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(len(label)):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
#    ax[1, idata].set_ylim(0, 10)

for idata in range(n_datasets):
    ax[2, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - over f + Linear.png')
plt.show()

In [ ]:
params_value_north = np.array(params_values_loglike, dtype=float)[0]
print(params_value_north.shape)

In [ ]:
discrete_values = [100, 200, 300]
colors = ['#EE6677', '#228833', '#4477AA']  

# Create a 1-row, 2-column subplot figure
fig = make_subplots(
    rows=1, 
    cols=2, 
    # subplot_titles=[
    #     "Noise of the North component data: \(f_{knee}\)",
    #     "Noise of the North component data: \(A_{white}\)"
    # ]
)

# ----------- First subplot: f_{knee} -----------
for val, col in zip(discrete_values, colors):
    mask = (distance_base_antenna2_test2 == val)
    
    x_vals = distance_antenna1_antenna2_test2[mask]
    # Here we take the column for f_{knee} (index 1)
    y_vals = params_value_north[int(distance_base_antenna1_test1.shape[0]):, 1][mask]
    
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='markers',
            marker=dict(size=10, color=col),
            name=f"{val} cm",
                        showlegend=False  # This removes the legend for the second subplot

        ),
        row=1,
        col=1
    )

# ----------- Second subplot: A_{white} -----------
for val, col in zip(discrete_values, colors):
    mask = (distance_base_antenna2_test2 == val)
    
    x_vals = distance_antenna1_antenna2_test2[mask]
    # Here we take the column for A_{white} (index 0)
    y_vals = params_value_north[int(distance_base_antenna1_test1.shape[0]):, 0][mask]
    
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='markers',
            marker=dict(size=10, color=col),
            name=f"{val} cm"
        ),
        row=1,
        col=2
    )

# Update axis labels for each subplot
fig.update_xaxes(title='Distance between antennas (cm)', row=1, col=1, gridcolor='lightgray')
fig.update_xaxes(title='Distance between antennas (cm)', row=1, col=2, gridcolor='lightgray')
fig.update_yaxes(title=r'$f_{knee}$', row=1, col=1, gridcolor='lightgray')
fig.update_yaxes(title=r'$A_{white}$', row=1, col=2, gridcolor='lightgray')

# Update overall layout
fig.update_layout(
    title='Noise parameters of the North component data',
    legend_title='Distance between base and antenna 2',
    width=1500,
    height=900,
    template='plotly_white',   # sets a clean background theme
    paper_bgcolor='white',     # ensures the outer area is white
    plot_bgcolor='white'       # ensures the plotting area is white
)
fig.write_image(f"Noise parameters with distance - over f + linear + white.png")
fig.show()